In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yaml
from pathlib import Path
from src.L_functions import get_vol, get_neg_sharpe, rast
from src.Models import Multi_CBO_model, gen_Psi,CBO_model
from src.utils import Lmin, avg_x, avg_L, load_config, next_run_index, save_run_config
import random
import os
import sys
from tqdm import trange  # pip install tqdm

In [2]:
def has_converged_all_coords_equal(X: np.ndarray, tol: float = 1e-6) -> bool:
    """
    X: (N, D) 파티클 행렬
    tol: 좌표별 허용 오차. max-min <= tol 이면 같은 값으로 간주.
    """
    # np.ptp = max - min, axis=0 → 각 좌표(l)별 범위
    return np.all(np.ptp(X, axis=0) <= tol)

In [3]:
RESULTS_ROOT = Path("./results")
SIMNAME_SUB  = "250928RAST"


# ---- 메인 ----
# np.random.seed(1)
base_cfg   = load_config("configs/configRAST.yaml")

for sigma in [0, 1, 2, 3]:
    run_idx  = next_run_index(RESULTS_ROOT, SIMNAME_SUB)
    run_name = f"{SIMNAME_SUB}_{run_idx}"
    run_root = RESULTS_ROOT / run_name           # 상위 simname 폴더 (여기 아래에 sim0000 ... 생성)
    run_root.mkdir(parents=True, exist_ok=True)

    # 원본 yaml 직접 덮지 말고 메모리에서 수정
    config = dict(base_cfg)
    config["sigma"]    = sigma
    config["simname"] = run_name
    save_run_config(run_root, config)

    # 문제 정의

    L = rast



    # 공용 초기값 (원하면 실험별 동일 초기값)
    x0 = 2 * np.random.rand(config['nump'], config['D']) + 2
    np.save(run_root / "x0.npy", x0)

    nsim    = int(config['sim'])
    maxiter = int(config.get('maxiter', 1000))

    # 진행률: 바깥 루프(시뮬레이션 개수)
    for simnum in range(nsim):
        # 개별 sim 폴더 생성 ...
        sim_dir  = run_root / f"sim{simnum:04d}"
        figs_dir = sim_dir / "figs"
        figs_dir.mkdir(parents=True, exist_ok=True)
    
        model = CBO_model(L, config)
        model.path = str(sim_dir.resolve()) + "/"
        model.make_path()
    
        x = np.load(run_root / "x0.npy")
        E = np.zeros(maxiter, dtype=float)
    
        for it in range(maxiter):
    
            if config.get('model', 1) == 1:
                x = model.step(x, L)
            else:
                raise NotImplementedError("model!=1 분기 사용 시 다른 모델 지정 필요")
    
            model.trace_func(x, lambda z: z, "coord")
            model.trace_func(x, lambda x: model.best_loss(x, L), "best_L")
            # --- 진행률 (print 버전) ---
            # if (it + 1) % 50 == 0 or it == maxiter - 1:
            #     pct = (simnum * maxiter + it + 1) / (nsim * maxiter) * 100
            #     print(f"\rProgress: {pct:.1f}%", end="")
            if has_converged_all_coords_equal(x,1e-4):
                print(it)
                break
        # 시뮬레이션 종료 후 개행
    
        model.save_func("coord", simnum=simnum)
        model.save_func("best_L", simnum=simnum)
        # eps = 1e-12
        # plt.figure()
        # plt.plot(np.arange(maxiter), E + eps, label='E_n')
        # plt.yscale('log')
        # plt.title(f'sim{simnum:04d}')
        # plt.xlabel('iter')
        # plt.ylabel('energy')
        # plt.legend()
        # plt.savefig(figs_dir / "energy.png", dpi=150, bbox_inches="tight")
        # plt.close()


14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
21
21
26
22
22
20
22
26
26
22
23
23
24
23
24
22
22
22
23
25
24
24
21
24
22
23
25
24
23
25
23
22
23
24
22
22
22
23
23
28
22
23
24
26
24
26
24
24
22
21
48
48
44
49
47
46
47
39
50
46
50
67
43
47
42
46
55
42
47
40
45
54
44
42
57
51
45
44
48
55
46
52
52
38
44
48
43
44
52
51
44
47
46
45
46
42
43
52
46
53
146
165
133
148
192
183
167
151
144
147
137
138
155
180
128
150
161
190
165
170
156
142
141
149
159
135
178
153
162
147
158
148
189
199
153
163
203
143
168
150
144
163
166
164
153
134
155
169
152
167


In [4]:
from src.trace_stats import summarize_scalar_trace, summarize_coord_trace, save_stats_json

run_root = "results/250928RAST_0"   # 본인 실험 폴더

# 1) best_L (스칼라) 통계
bestL_stats = summarize_scalar_trace(run_root, funcname="best_L")
print("best_L mean[-10:]:", bestL_stats["mean"][-10:])
save_stats_json(bestL_stats, f"{run_root}/stats/best_L_stats.json")

# 2) coord (N,D) 통계
coord_stats = summarize_coord_trace(run_root, funcname="coord",
                                    reduce_over="particles_and_sims")
print("coord mean shape:", coord_stats["mean"].shape)  # (T, D)
save_stats_json(coord_stats, f"{run_root}/stats/coord_stats.json")

best_L mean[-10:]: [13.40474483 13.40150251 13.40293739 13.40653523 13.41034863 13.41307959
 13.41468792 13.41555604 13.41600638 13.41623565]
coord mean shape: (15, 15)


In [5]:
bestL_stats

{'iters': 15,
 'n_sims': 50,
 'mean': array([15.03670265, 13.64904321, 13.44605639, 13.44605639, 13.4161037 ,
        13.40474483, 13.40150251, 13.40293739, 13.40653523, 13.41034863,
        13.41307959, 13.41468792, 13.41555604, 13.41600638, 13.41623565]),
 'var': array([0.00000000e+00, 1.13595970e-28, 7.88860905e-29, 7.88860905e-29,
        2.83989926e-29, 3.15544362e-30, 3.15544362e-30, 1.26217745e-29,
        3.15544362e-28, 3.15544362e-28, 3.15544362e-28, 1.26217745e-29,
        7.88860905e-29, 1.26217745e-29, 3.15544362e-30]),
 'std': array([0.00000000e+00, 1.06581410e-14, 8.88178420e-15, 8.88178420e-15,
        5.32907052e-15, 1.77635684e-15, 1.77635684e-15, 3.55271368e-15,
        1.77635684e-14, 1.77635684e-14, 1.77635684e-14, 3.55271368e-15,
        8.88178420e-15, 3.55271368e-15, 1.77635684e-15])}